SqlAlchemy helps to proceed logic of data manipulation 
when we handle interaction with user



We have three main points


![Illustration](.excalidraw.png)






In [2]:
from typing import Optional, List
from sqlalchemy import ForeignKey
from sqlalchemy.orm import Mapped, mapped_column,declarative_base
from sqlalchemy.orm import relationship, backref
from datetime import datetime
from dataclasses import dataclass

Base = declarative_base()



class User(Base):
    __tablename__ = "user"

    id: Mapped[int]= mapped_column(primary_key=True)
    name: Mapped[str] 
    mipt_mail: Mapped[Optional[str]] = mapped_column(default='') 
    admin: Mapped[Optional[bool]] = mapped_column(default=False)
    homeworks: Mapped[List["UserHomeworkResult"]] = relationship(backref=backref('user'),cascade='all, delete-orphan')

    def __repr__(self):
        return (
            f'Telegram id: {self.id}'
            f'Name: {self.name}' 
            f'Mipt_mail: {self.mipt_mail}'
        )



class Homework(Base):
    __tablename__ = 'homework'
    id: Mapped[int] = mapped_column(primary_key = True) 
    week: Mapped[int]
    description: Mapped[Optional[str]]
    homework_link: Mapped[Optional[str]]

    def __repr__(self):
        return (
            f'Homework id: {self.id}'
            f'Week: {self.week}'
            f'Description: {self.description}'
            f'Homework Link: {self.homework_link}'
        )


class UserHomeworkResult(Base):
    __tablename__ = 'homework_results'
 
    id: Mapped[int] = mapped_column(primary_key = True) 
    homework_id : Mapped[int] = mapped_column(ForeignKey("homework.id")) 
    user_id: Mapped[int] = mapped_column(ForeignKey("user.id")) 
    student_homework_link: Mapped[Optional[str]]
    result: Mapped[int] = mapped_column(default=0)
    valid_from_dttm: Mapped[datetime] = mapped_column(default=datetime.now())
    homework: Mapped["Homework"] = relationship(backref='results')


    def __repr__(self):
        return (
            f'Homework id: {self.homework_id}'
            f'User id: {self.user_id}'
            f'student_homework_link: {self.student_homework_link}'
            f'result: {self.result}'
            f'Valid_from_dttm: {self.valid_from_dttm}'
            f'Homework :{self.homework}'
        )


ArgumentError: Invalid cascade option(s): 'delete_orphan'

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
# база будет создана в памяти
engine = create_engine('sqlite://')
Base.metadata.create_all(engine)


In [3]:
from sqlalchemy.orm import Session

# add_homework

with Session(engine) as session:
    hw1 = Homework(
        week = 1,
        description = 'Работа с SQlAlchemy',
        homework_link = 'github.com'
    )
    hw2 = Homework(
        week = 2,
        description = 'Продвинутая работа с SQlAlchemy',
        homework_link = 'github2.com'
    )

    session.add_all([hw1,hw2])
    
    session.commit()

ArgumentError: For many-to-one relationship UserHomeworkResult.user, delete-orphan cascade is normally configured only on the "one" side of a one-to-many relationship, and not on the "many" side of a many-to-one or many-to-many relationship.  To force this relationship to allow a particular "User" object to be referred towards by only a single "UserHomeworkResult" object at a time via the UserHomeworkResult.user relationship, which would allow delete-orphan cascade to take place in this direction, set the single_parent=True flag. (Background on this error at: https://sqlalche.me/e/20/bbf0)

In [7]:
from sqlalchemy import select
with Session(engine) as session:
    stmt = select(Homework)
    for ent in session.execute(stmt).scalars():
        print(ent.results)

[]
[]


In [8]:
from sqlalchemy import select
with Session(engine) as session:
    stmt = select(Homework)
    homeworks = session.execute(stmt).scalars()
    new_user = User(id=1,name='Nikita')
    new_user.homeworks = [UserHomeworkResult(homework=hw) for hw in homeworks] 
    session.merge(new_user)
    session.commit()

/tmp/ipykernel_121291/209486939.py:7: SAWarning: Object of type <UserHomeworkResult> not in session, add operation along 'Homework.results' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  session.merge(new_user)


In [13]:
from sqlalchemy import select
with Session(engine) as session:
    stmt = select(User).where(User.id == 1)
    user: User = session.execute(stmt).scalars().first()
    print(user)
    for hw in user.homeworks:
        print(hw)
    user.homeworks = [user.homeworks[0]]
    session.merge(user)
    session.commit()

Telegram id: 1Name: NikitaMipt_mail: 
Homework id: 1User id: 1student_homework_link: Noneresult: 0Valid_from_dttm: 2023-09-24 21:18:50.679980Homework :Homework id: 1Week: 1Description: Работа с SQlAlchemyHomework Link: github.com
Homework id: 2User id: 1student_homework_link: Noneresult: 0Valid_from_dttm: 2023-09-24 21:18:50.679980Homework :Homework id: 2Week: 2Description: Продвинутая работа с SQlAlchemyHomework Link: github2.com


IntegrityError: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) NOT NULL constraint failed: homework_results.user_id
[SQL: UPDATE homework_results SET user_id=? WHERE homework_results.id = ?]
[parameters: (None, 2)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)